In [1]:
!pip install keras-tuner

You should consider upgrading via the 'c:\users\ved\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


#### California Housing Dataset

###### IMPORTING LIBRARY

In [2]:
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import kerastuner as kt

c:\Users\VED\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if sys.path[0] == '':


###### READING DATASET TRAIN/TEST

In [3]:
train_file_path = "california_housing_train.csv"
test_file_path = "california_housing_test.csv"

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
display(train_df.head(3))
display(test_df.head(3))

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0


###### SHAPE OF DATASET

In [4]:
display(train_df.shape)
display(test_df.shape)

(17000, 9)

(3000, 9)

###### DATASET DESCRIPTION OF TRAIN/TEST

In [5]:
display(train_df.describe())
display(test_df.describe())

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000,3000.00000
mean,-119.589200,35.63539,28.845333,2599.578667,529.950667,1402.798667,489.91200,3.807272,205846.27500
std,1.994936,2.12967,12.555396,2155.593332,415.654368,1030.543012,365.42271,1.854512,113119.68747
min,-124.180000,32.56000,1.000000,6.000000,2.000000,5.000000,2.00000,0.499900,22500.00000
25%,-121.810000,33.93000,18.000000,1401.000000,291.000000,780.000000,273.00000,2.544000,121200.00000
50%,-118.485000,34.27000,29.000000,2106.000000,437.000000,1155.000000,409.50000,3.487150,177650.00000
75%,-118.020000,37.69000,37.000000,3129.000000,636.000000,1742.750000,597.25000,4.656475,263975.00000
max,-114.490000,41.92000,52.000000,30450.000000,5419.000000,11935.000000,4930.00000,15.000100,500001.00000


##### CHECK NULL VALUES

In [7]:
display(train_df.isnull().sum())
display(test_df.isnull().sum())

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

##### SPLIT DATASET INTO DEPENDEND AND INDEPENDENT FEATURE

In [8]:
TARGET_NAME = "median_house_value"

X_train, y_train = train_df.drop(TARGET_NAME, axis=1), train_df[TARGET_NAME]
X_test, y_test = test_df.drop(TARGET_NAME, axis=1), test_df[TARGET_NAME]

#### FEATURE SCALING

In [9]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### KERAS TUNER



An open-source Python library called Keras Tuner was created specifically for adjusting the hyperparameters of artificial neural networks. Currently, the Keras tuner supports four different types of tuners or algorithms:

    Bayesian Optimization
    Hyperband
    Sklearn
    Random Search

The Hyper model is the one you want to tune.

    Using the model-building feature we can enable this tuning feature

We are going to build the ann with 4 layers

    Input layer
    Hidden layer 1
    Hidden layer 2
    Output Layer

unit1, unit2, unit3 will have the range between 25 to 512 with step 32

These will be apply to input layer and hidden layers

The learning rate will be picked up based on given inputs and hence we are use choice method to perform that action

Since this is the regression problem, we are using 'linear' activation function in the output layer. on the hidden layers we are using 'relu'.

For weight updation, we are using adam optimizers and the loss will be MSLE (Mean Squarred Log Error)


In [10]:
from sklearn import metrics


msle = MeanSquaredLogarithmicError()

def buildANNModel(hp):
    model = tf.keras.Sequential()
    
    units1 = hp.Int('units1', min_value=25, max_value=512, step=32)
    units2 = hp.Int('units2', min_value=25, max_value=512, step=32)
    units3 = hp.Int('units3', min_value=25, max_value=512, step=32)
    
    model.add(Dense(units=units1, activation='relu'))
    model.add(Dense(units=units2, activation='relu'))
    model.add(Dense(units=units3, activation='linear'))
    
    hp_learning_rate = hp.Choice(
        'learning_rate', 
        values=[1e-2, 1e-3, 1e-4]
        )
    
    model.compile(
        optimizer = Adam(learning_rate=hp_learning_rate),
        loss=msle,
        metrics=[msle]
    )
    
    return model


In order to get a high-performing model quickly, the Hyperband tuning algorithm makes use of adaptive resource allocation and early-stopping.

In [12]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    buildANNModel,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

In [14]:
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

Trial 30 Complete [00h 01m 02s]
val_mean_squared_logarithmic_error: 7.535202503204346

Best val_mean_squared_logarithmic_error So Far: 0.381743460893631
Total elapsed time: 00h 11m 00s
INFO:tensorflow:Oracle triggered exit


In [15]:
for p in ['units1','units2','units3','learning_rate']:
  print(p, tuner.get_best_hyperparameters()[0].get(p))

units1 281
units2 345
units3 25
learning_rate 0.01


#### MODEL EVALUATION

In [16]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [17]:
model = tuner.hypermodel.build(best_hps)

In [19]:
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
425/425 [==============================] - 3s 5ms/step - loss: 8.8295 - mean_squared_logarithmic_error: 8.8295 - val_loss: 2.5687 - val_mean_squared_logarithmic_error: 2.5528
Epoch 2/50
425/425 [==============================] - 2s 5ms/step - loss: 0.5676 - mean_squared_logarithmic_error: 0.5676 - val_loss: 0.9957 - val_mean_squared_logarithmic_error: 0.9899
Epoch 3/50
425/425 [==============================] - 3s 6ms/step - loss: 0.3127 - mean_squared_logarithmic_error: 0.3127 - val_loss: 0.6133 - val_mean_squared_logarithmic_error: 0.6115
Epoch 4/50
425/425 [==============================] - 3s 6ms/step - loss: 0.2487 - mean_squared_logarithmic_error: 0.2487 - val_loss: 0.4532 - val_mean_squared_logarithmic_error: 0.4536
Epoch 5/50
425/425 [==============================] - 2s 5ms/step - loss: 0.2060 - mean_squared_logarithmic_error: 0.2060 - val_loss: 0.3560 - val_mean_squared_logarithmic_error: 0.3580
Epoch 6/50
425/425 [==============================] - 2s 5ms/step - lo

In [20]:
eval_result = model.evaluate(X_test, y_test)
print(eval_result)

94/94 [==============================] - 0s 4ms/step - loss: 0.0921 - mean_squared_logarithmic_error: 0.0921
[0.09214451909065247, 0.09213564544916153]


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 281)               2529      
_________________________________________________________________
dense_4 (Dense)              (None, 345)               97290     
_________________________________________________________________
dense_5 (Dense)              (None, 25)                8650      
Total params: 108,469
Trainable params: 108,469
Non-trainable params: 0
_________________________________________________________________
